<a href="https://colab.research.google.com/github/Zarawar5555/WeatherWise/blob/main/AITensorFlowLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas scikit-learn tensorflow numpy
import pandas as pd
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))
df.head()
from sklearn.preprocessing import LabelEncoder
import numpy as np
condition_encoder = LabelEncoder()
df['condition_enc'] = condition_encoder.fit_transform(df['condition'])
suggestion_encoder = LabelEncoder()
df['suggestion_enc'] = suggestion_encoder.fit_transform(df['suggestion'])


X = df[['outdoor_temp', 'outdoor_humidity', 'indoor_temp', 'indoor_humidity', 'condition_enc']].values
y = df['suggestion_enc'].values

from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)
import tensorflow as tf
from tensorflow.keras import layers, models
model = models.Sequential([
    layers.Input(shape=(5,)),
    layers.Dense(8, activation='relu'),
   layers.Dense(y_cat.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("comfort_model.tflite", "wb") as f:
    f.write(tflite_model)
print("✅ TFLite model saved as 'comfort_model.tflite'")
from google.colab import files
files.download("comfort_model.tflite")
